In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import random
import pandas as pd
import numpy as np
import itertools

### homework_ex2_01

In [2]:
data1 = pd.read_csv("../data/ex2data1.txt", names=['exam1', 'exam2', 'accepted'])
X1 = np.array(data1.iloc[:, 0:-1])
y1 = np.array(data1.iloc[:, -1]).reshape((-1, 1))

In [3]:
class LogisticRegression:
    def __init__(self, num_inputs, lr=0.01, batch_size=32, num_epochs=200000):
        self.w = torch.zeros((num_inputs, 1), requires_grad=True)
        self.b = torch.zeros(1, requires_grad=True)
        self.lr = lr
        self.batch_size = batch_size
        self.num_epochs = num_epochs
        self.params = [self.w, self.b]
        self.loss_fn = nn.BCELoss()

    def net(self, X):
        """逻辑回归模型"""
        return torch.matmul(X, self.w) + self.b

    def sgd(self):
        """小批量随机梯度下降"""
        with torch.no_grad():
            for param in self.params:
                param -= self.lr * param.grad / self.batch_size
                param.grad.zero_()

    def fit(self, features, labels):
        """训练模型"""
        for epoch in range(self.num_epochs):
            for X, y in self.data_iter(self.batch_size, features, labels):
                logits = self.net(X)
                l = self.loss_fn(torch.sigmoid(logits), y)
                l.backward()
                self.sgd()
            if (epoch + 1) % 20000 == 0:
                with torch.no_grad():
                    train_l = self.loss_fn(torch.sigmoid(self.net(features)), labels)
                    print(f"Epoch {epoch + 1}, Loss: {train_l.item()}")  # 直接使用 .item()

    @staticmethod
    def data_iter(batch_size, features, labels):
        """数据迭代器"""
        num_examples = len(features)
        indices = list(range(num_examples))
        random.shuffle(indices)
        for i in range(0, num_examples, batch_size):
            batch_indices = torch.tensor(indices[i: min(i + batch_size, num_examples)])
            yield features[batch_indices], labels[batch_indices]

In [4]:
features = torch.tensor(X1, dtype=torch.float32)
labels = torch.tensor(y1, dtype=torch.float32)

model = LogisticRegression(num_inputs=features.shape[1])
model.fit(features, labels)

threshold = 0.5
with torch.no_grad():
    logits = model.net(features)
    probs = torch.sigmoid(logits)
    preds = (probs >= threshold).float()
    correct = preds.eq(labels).sum().item()
    total = labels.size(0)
    accuracy_torch = correct / total
    print(f'Accuracy: {accuracy_torch}')

Epoch 20000, Loss: 0.5374067425727844
Epoch 40000, Loss: 0.4680849313735962
Epoch 60000, Loss: 0.42477360367774963
Epoch 80000, Loss: 0.3883751928806305
Epoch 100000, Loss: 0.3723139464855194
Epoch 120000, Loss: 0.34814947843551636
Epoch 140000, Loss: 0.35725048184394836
Epoch 160000, Loss: 0.3253430128097534
Epoch 180000, Loss: 0.3102661371231079
Epoch 200000, Loss: 0.2988196313381195
Accuracy: 0.89


### homework_ex2_02

In [5]:
data2 = pd.read_csv("../data/ex2data2.txt", names=['test1', 'test2', 'accepted'])
X2 = np.array(data2.iloc[:, 0:-1])
y2 = np.array(data2.iloc[:, -1]).reshape(-1, 1)

def polynomial_feature_map(X, max_degree=6):
    n_samples, n_features = X.shape
    features = []
    for degree in range(1, max_degree + 1):
        for items in itertools.combinations_with_replacement(range(n_features), degree):
            features.append(np.prod(X[:, items], axis=1))
    features.append(np.ones(n_samples))
    return np.column_stack(features)

X_map = polynomial_feature_map(X2)

In [6]:
class LogisticRegression(nn.Module):
    def __init__(self, in_features):
        super(LogisticRegression, self).__init__()
        self.linear = nn.Linear(in_features, 1)

    def forward(self, x):
        return torch.sigmoid(self.linear(x))

    @staticmethod
    def data_iter(features, labels, batch_size=16):
        num_examples = len(features)
        indices = list(range(num_examples))
        random.shuffle(indices)
        for i in range(0, num_examples, batch_size):
            batch_indices = torch.tensor(indices[i: min(i + batch_size, num_examples)], dtype=torch.long)
            yield features[batch_indices], labels[batch_indices]

In [7]:
features = torch.tensor(X_map, dtype=torch.float32)
labels = torch.tensor(y2, dtype=torch.float32)
model = LogisticRegression(in_features=features.shape[1])
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

epochs = 100000

for epoch in range(epochs):
    total_loss = 0
    for X, y in model.data_iter(features, labels):
        output = model(X)
        y = y.reshape(-1, 1)
        loss = criterion(output, y)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    with torch.no_grad():
        if (epoch + 1) % 10000 == 0:
            train_l = criterion(model(features), labels)
            print(f'Epoch {epoch + 1}/{epochs}, Loss: {train_l:.4f}')

threshold = 0.5
with torch.no_grad():
    probs = model(features).squeeze()
    preds = (probs >= threshold).float()
    labels = labels.squeeze()
    correct = preds.eq(labels).sum().item()
    total = labels.size(0)
    accuracy_torch = correct / total
    print(f'Accuracy: {accuracy_torch}')

Epoch 10000/100000, Loss: 0.3093
Epoch 20000/100000, Loss: 0.3018
Epoch 30000/100000, Loss: 0.2985
Epoch 40000/100000, Loss: 0.2965
Epoch 50000/100000, Loss: 0.2949
Epoch 60000/100000, Loss: 0.2937
Epoch 70000/100000, Loss: 0.2925
Epoch 80000/100000, Loss: 0.2916
Epoch 90000/100000, Loss: 0.2907
Epoch 100000/100000, Loss: 0.2899
Accuracy: 0.864406779661017
